# Deterministic forecasting of the gold price using ARIMA method

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pmdarima as pm
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Get gold price df and set date as index
df_au = pd.read_csv("../data/AU.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

## Preparing the time series to fit the ARIMA models

### Resample to weekly prices

In [ ]:
# Resample to weekly prices (taking the mean of the week)
df_au = df_au.resample("W").mean()
df_au = df_au.asfreq("W")
# Front fill missing values
df_au.ffill(inplace=True)

### Split data into train and test

In [ ]:
# Split the data into train and test sets based on date
split_date = pd.Timestamp("2022-01-01")
df_au_train = df_au.loc[df_au.index < split_date].copy()
df_au_test = df_au.loc[df_au.index >= split_date].copy()

In [ ]:
# Check for missing values
print(df_au_train.isnull().sum())
print(df_au_test.isnull().sum())

### Check if the time series is stationary

In [ ]:
# Plot the df
df_au_train.plot()

The plot clearly shows an upward trend, indicating the series is not stationary. Let's confirm this with other methods.

#### Autocorrelation function (ACF) and partial autocorrelation function (PACF)

In [ ]:
# Plot the acf and pacf
acf = plot_acf(df_au_train)
pacf = plot_pacf(df_au_train)

- The autocorrelation function (ACF) plot shows that the correlations decline slowly, indicating a strong and persistent correlation over time. This slow decay suggests that the time series data is non-stationary.
- The partial autocorrelation function (PACF) plot reveals that only the lag 1 value is significantly different from zero, indicating a strong correlation at lag 1 but not beyond. This pattern is typical of an autoregressive process.
- Together, these observations suggest that the time series data is non-stationary and shows a high degree of autocorrelation.

#### Augmented Dickey–Fuller (ADF) test

In [ ]:
# Perform ADF test
adf_test  = adfuller(df_au_train)
print(f"p-value: {adf_test[1]}")

- The ADF test results in a p-value close to 1, which is significantly greater than the 0.05 threshold.
- A high p-value indicates that we fail to reject the null hypothesis of the ADF test, which states that the time series has a unit root (non-stationary)

### Transform time series into stationary using differencing

In [ ]:
# Create diff from df
df_au_train_diff = df_au_train.diff().dropna()

### Check again for stationarity and determine ARIMA parameters

In [ ]:
# Plot the acf and pacf
acf = plot_acf(df_au_train_diff)
pacf = plot_pacf(df_au_train_diff)

In [ ]:
# Perform ADF test
adf_test  = adfuller(df_au_train_diff)
print(f"p-value: {adf_test[1]}")

- After differencing, we can conclude that the time series is stationary with "d" = 1.
- Based on the ACF and PACF plots, "q" is 1 and "p" is 1.
- Let's use the pmdarima module to obtain the "auto" parameters.

### Determine the ARIMA parameters using pmdarima

In [ ]:
model_auto = pm.auto_arima(df_au_train, seasonal=True, m=52, trace=True, error_action="ignore", suppress_warnings=True, stepwise=True)
model_auto.summary()

## Fit the ARIMA models

### Train the ARIMA models

#### Manual model

In [ ]:
manual_model = ARIMA(df_au_train, order=(1, 1, 1))
manual_model_fit = manual_model.fit()
manual_model_fit.summary()

#### Auto model

In [ ]:
# Auto model is seasonal
auto_model = ARIMA(df_au_train, order=model_auto.order, seasonal_order=model_auto.seasonal_order)
auto_model_fit = auto_model.fit()
auto_model_fit.summary()

### Check models residuals

#### Manual model

In [ ]:
residuals = manual_model_fit.resid[1:]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_acf(residuals)
plot_pacf(residuals)
residuals.plot(title="Residuals", ax=axes[0])
residuals.plot(title="Density", ax=axes[1], kind="kde")

#### Auto model

In [ ]:
residuals = auto_model_fit.resid[1:]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_acf(residuals)
plot_pacf(residuals)
residuals.plot(title="Residuals", ax=axes[0])
residuals.plot(title="Density", ax=axes[1], kind="kde")

- Residuals resemble white noise in both models.
- Their density is normally distributed.
- ACF and PACF plots show no significant spikes for both models.
- Therefore, the models appear to be well-fitted and meet the necessary diagnostic criteria.

### Forecast on the training data

In [ ]:
# Get number of periods to forecast from test
n_forecast = len(df_au_test)

#### Manual model

In [ ]:
# Get forecast values
manual_forecast = manual_model_fit.get_forecast(steps=n_forecast)
manual_forecast_values = manual_forecast.predicted_mean
manual_conf_int = manual_forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the training data
plt.plot(df_au_train.loc["2020-01-01":].index, df_au_train.loc["2020-01-01":].values, label="Train")

# Plot the test data
plt.plot(df_au_test.loc["2020-01-01":].index, df_au_test.loc["2020-01-01":].values, label="Test")

# Plot the forecast data
plt.plot(df_au_test.loc["2020-01-01":].index, manual_forecast_values, label="Forecast")

# Fill the confidence interval
plt.fill_between(df_au_test.loc["2020-01-01":].index, manual_conf_int.iloc[:, 0], manual_conf_int.iloc[:, 1], color="pink", alpha=0.3)

plt.title("ARIMA forecast of the gold price")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper left")

#### Auto model

In [ ]:
# Get forecast values
auto_forecast = auto_model_fit.get_forecast(steps=n_forecast)
auto_forecast_values = auto_forecast.predicted_mean
auto_conf_int = auto_forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the training data
plt.plot(df_au_train.loc["2020-01-01":].index, df_au_train.loc["2020-01-01":].values, label="Train")

# Plot the test data
plt.plot(df_au_test.loc["2020-01-01":].index, df_au_test.loc["2020-01-01":].values, label="Test")

# Plot the forecast data
plt.plot(df_au_test.loc["2020-01-01":].index, auto_forecast_values, label="Forecast")

# Fill the confidence interval
plt.fill_between(df_au_test.loc["2020-01-01":].index, auto_conf_int.iloc[:, 0], auto_conf_int.iloc[:, 1], color="pink", alpha=0.3)

plt.title("SARIMA forecast of the gold price")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper left")

### Evaluate both models

#### Manual model

In [ ]:
# Evaluate model
mae = mean_absolute_error(df_au_test, manual_forecast_values)
rmse = np.sqrt(mean_squared_error(df_au_test, manual_forecast_values))
r2 = r2_score(df_au_test, manual_forecast_values)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

#### Auto model

In [ ]:
# Evaluate model
mae = mean_absolute_error(df_au_test, auto_forecast_values)
rmse = np.sqrt(mean_squared_error(df_au_test, auto_forecast_values))
r2 = r2_score(df_au_test, auto_forecast_values)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

**Unfortunately, neither the manual nor the auto models provided acceptable results. We need to try a different strategy.**

### Fine tune a model

In [ ]:
# Fit tuned model
tuned_model = ARIMA(df_au_train, order=(3, 1, 0), seasonal_order=(0, 1, 1, 52))
tuned_model_fit = tuned_model.fit()
tuned_model_fit.summary()

In [ ]:
# Get forecast values
tuned_forecast = tuned_model_fit.get_forecast(steps=n_forecast)
tuned_forecast_values = tuned_forecast.predicted_mean
tuned_conf_int = tuned_forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the training data
plt.plot(df_au_train.loc["2020-01-01":].index, df_au_train.loc["2020-01-01":].values, label="Train")

# Plot the test data
plt.plot(df_au_test.loc["2020-01-01":].index, df_au_test.loc["2020-01-01":].values, label="Test")

# Plot the forecast data
plt.plot(df_au_test.loc["2020-01-01":].index, tuned_forecast_values, label="Forecast")

# Fill the confidence interval
plt.fill_between(df_au_test.loc["2020-01-01":].index, tuned_conf_int.iloc[:, 0], tuned_conf_int.iloc[:, 1], color="pink", alpha=0.3)

plt.title("SARIMA forecast of the gold price")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper left")

In [ ]:
# Evaluate model
mae = mean_absolute_error(df_au_test, tuned_forecast_values)
rmse = np.sqrt(mean_squared_error(df_au_test, tuned_forecast_values))
r2 = r2_score(df_au_test, tuned_forecast_values)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

**This model has provided acceptable results. Therefore it'll be used for the forecast.**

## The forecast

In [ ]:
# Fit model
model = ARIMA(df_au, order=(3, 1, 0), seasonal_order=(0, 1, 1, 52))
model_fit = model.fit()
model_fit.summary()

In [ ]:
# Get number of periods (weeks) to forecast
n_forecast = 26

In [ ]:
# Get forecast values
forecast = model_fit.get_forecast(steps=n_forecast)
forecast_values = forecast.predicted_mean
conf_int = forecast.conf_int()
forecast_index = pd.date_range(start=df_au.index[-1], periods=n_forecast + 1, freq="W")[1:]

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the original data
sns.lineplot(x=df_au.loc["2022-01-01":].index, y=df_au.loc["2022-01-01":]["price"], label="Historic", color="yellow")

# Plot the forecast data
sns.lineplot(x=forecast_index, y=forecast_values, label="Forecast", color="aqua")

# Fill the confidence interval
plt.fill_between(forecast_index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], label="Confidence interval", color="aquamarine", alpha=0.2)

plt.title("SARIMA forecast of the gold price for the next 6 months")
plt.xlabel("")
plt.ylabel("")
plt.legend(loc="upper left")

save_chart_as_png("7.1_AU_ARIMA")